## HuReTEx DLM 0.01 (2025.03.13)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import models
from keras import layers
from keras.callbacks import ModelCheckpoint
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
from interface import implements
import importlib
import ipynb

2025-08-14 21:25:32.665409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-14 21:25:32.665437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-14 21:25:32.666285: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-14 21:25:32.670655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-14 21:25:33.182384: W tensorflow/compiler/tf2

In [10]:
from ipynb.fs.full.HuReTEx_DLSI_0_01 import DeepLearningSystemInterface
importlib.reload(ipynb.fs.full.HuReTEx_DLSI_0_01)

<module 'ipynb.fs.full.HuReTEx_DLSI_0_01' (/var/srv/183110/Moje/Nauka/HURETEX/HURETEX/HuReTEx_DLSI_0_01.ipynb)>

In [ ]:
class SimpleCDLS4MNIST(implements(DeepLearningSystemInterface)):

    def __init__(self):

        self.n_filters_conv_1 = 4
        self.filter_size_conv_1 = 5
        self.n_filters_conv_2 = 4
        self.filter_size_conv_2 = 5
        self.n_neurons_dense_1 = 50
        self.n_neurons_dense_2 = 10

        (x_train_valid, y_train_valid), (x_test, y_test) = mnist.load_data()
        x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, stratify=y_train_valid)

        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
        x_valid = x_valid.reshape((x_valid.shape[0], x_valid.shape[1], x_valid.shape[2], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

        x_train = x_train.astype('float32')
        x_valid = x_valid.astype('float32')
        x_test = x_test.astype('float32')

        self.x_train = x_train/255.0
        self.x_valid = x_valid/255.0
        self.x_test = x_test/255.0

        self.y_train = tf.keras.utils.to_categorical(y_train)
        self.y_valid = tf.keras.utils.to_categorical(y_valid)
        self.y_test = tf.keras.utils.to_categorical(y_test)

        self.model = models.Sequential()
        self.model.add(layers.Conv2D(self.n_filters_conv_1, (self.filter_size_conv_1, self.filter_size_conv_1), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3])))
        self.model.add(layers.Conv2D(self.n_filters_conv_2, (self.filter_size_conv_2, self.filter_size_conv_2), activation='relu'))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(self.n_neurons_dense_1, activation='relu'))
        self.model.add(layers.Dense(self.n_neurons_dense_2, activation='softmax'))
        self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    def train_model(self):

        n_epochs = 10
        batch_size = 16

        checkpoint = ModelCheckpoint('best_model.keras', 
            verbose=1, 
            monitor='val_loss',
            save_best_only=True, 
            mode='auto'
        ) 

        self.model.fit(self.x_train, self.y_train, epochs=n_epochs, batch_size=batch_size, validation_data=(self.x_valid, self.y_valid),
                       callbacks=[checkpoint])

    def calculate_activations(self):

        layer_outputs = [layer.output for layer in self.model.layers[:5]]
        activation_model = models.Model(inputs=self.model.layers[0].input, outputs=layer_outputs)
        self.activations = activation_model.predict(x=self.x_train)

    def calculate_artifact_clusters(self):

        self.n_clusters_conv_1 = 4
        self.n_clusters_conv_2 = 4
        self.n_clusters_dense = 4

        self.artifact_clusters = pd.DataFrame()

        """ activations_f = self.activations[0][:,:,:,0]
        activations_f = activations_f.reshape([activations_f.shape[0],activations_f.shape[1]*activations_f.shape[2]])
        print(activations_f)
        np.savetxt("activations_f.csv", activations_f, delimiter=",") """

        for f in range(self.n_filters_conv_1):

            print('filter: '+str(f))

            activations_f = self.activations[0][:,:,:,f]
            activations_f = activations_f.reshape([activations_f.shape[0],activations_f.shape[1]*activations_f.shape[2]])
            ac_f = AgglomerativeClustering(n_clusters=self.n_clusters_conv_1).fit(activations_f)

            self.artifact_clusters['l0_f'+str(f)] = ac_f.labels_

        for f in range(self.n_filters_conv_2):

            print('filter: '+str(f))

            activations_f = self.activations[1][:,:,:,f]
            activations_f = activations_f.reshape([activations_f.shape[0],activations_f.shape[1]*activations_f.shape[2]])
            ac_f = AgglomerativeClustering(n_clusters=self.n_clusters_conv_2).fit(activations_f) 

            self.artifact_clusters['l1_f'+str(f)] = ac_f.labels_

        activations_d_1 = self.activations[3]
        ac_d = AgglomerativeClustering(n_clusters=self.n_clusters_dense).fit(activations_d_1)
        self.artifact_clusters['l3'] = ac_d.labels_

        predictions = self.model.predict(x=self.x_train)
        pred = np.argmax(predictions, axis=1)
        self.artifact_clusters['p'] = pred

    def get_sequential_information_system(self):

        self.sis = pd.DataFrame()
        #self.sis['conv1'] = self.artifact_clusters['l0_f'+filter_id]
        self.sis['conv1'] = self.artifact_clusters.astype(str).loc[:,['l0_f0', 'l0_f1', 'l0_f2', 'l0_f3']].apply('_'.join, axis=1)
        self.sis['conv2'] = self.artifact_clusters.astype(str).loc[:,['l1_f0', 'l1_f1', 'l1_f2', 'l1_f3']].apply('_'.join, axis=1)
        self.sis['dense1'] = self.artifact_clusters['l3']
        self.sis['output'] = self.artifact_clusters['p']

        return self.sis